# Introduction

This worksheet covers linear regression. Similar to last week, you will do some work implementing your own versions of these algorithms, to ensure that you understand the details of them. You will also compare them with the implementations in scikit-learn to test your implementations.

The box plots show that there is no clear difference in performance between the two models

## Linear Regression
In linear regression we make the assumption that the data $(x_i, y_i)$ can be modelled by a function of the form
$$ \hat{y_i} = f(\vec{x}_i)= \sum_j a_j x_{ij}  + b_i$$

Recall that we can express this in a matrix format by:
$$ \hat{\vec{y}} = f(X)= X\Theta$$

where
$$ X=\begin{pmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,n} &1 \\
\vdots & \vdots & \ldots & \vdots & \vdots \\
x_{N,1} & x_{N,2} & \ldots & x_{N,n} & 1
\end{pmatrix}, \quad \vec{y}=\begin{pmatrix} y_1 \\ \vdots \\y_N \end{pmatrix}, \quad \Theta=\begin{pmatrix} a_1 \\ \vdots \\a_n\\b \end{pmatrix}$$

We saw in lectures that the optimal value of $\Theta$ is given by setting
$$ \Theta = (X^T X)^{-1} X^T \vec{y}$$

The quantity $(X^T X)^{-1} X^T$ is called the psuedoinverse of X, and can be computed using the function `np.linalg.pinv`.

We will (a) perform a linear regression on the diabetes dataset. You can load this dataset using the function `load_diabetes` from `sklearn.datasets`. (b) compute the mean squared error and the R^2, and (c) compare your results with the built in function in sklearn (`sklearn.linear_model.LinearRegresion()`). You should get the same results.

In [ ]:
# import statments here
import math
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Part (a) Implementing linear regression

In [ ]:
# Load the diabetes dataset
diabetes = datasets.load_diabetes()
X = diabetes.data
Y = diabetes.target

# Split the dataset into training, validation and test, using test_size=0.2
Xtr, Xtest, Ytr, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtr, Ytr, test_size=0.2, random_state=0)

In [ ]:
# Add a column of ones to your Xtrain and Xtest for the intercept term
Wtrain=np.hstack([Xtrain, np.ones((len(Xtrain),1))])
Wval=np.hstack([Xval, np.ones((len(Xval),1))])
Wtest=np.hstack([Xtest, np.ones((len(Xtest),1))])

In [ ]:
# Calculate the value of the coefficients theta. You can use the function np.linalg.pinv
theta = np.linalg.pinv(Wtrain).dot(Ytrain)

## Part (b) Computing performance metrics

In [ ]:
# Make a prediction on the test set by applying the coefficients theta to the test set
my_pred = Wval.dot(theta)

In [ ]:
# Calculate the mean squared error and the R^2.
# You can use the built in functions from sklearn
print(f'mean squared error= {mean_squared_error(my_pred, Yval):.4f}')
print(f'R^2={r2_score(my_pred, Yval):.4f}')

## Part (c) Checking results
Compare your results with the built in function `sklearn.linear_model.LinearRegression()`

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(Xtrain, Ytrain)
pred = regr.predict(Xval)
print(f'mean squared error= {mean_squared_error(pred, Yval):.4f}')
print(f'R^2={r2_score(pred, Yval):.4f}')

Visualise the perfomance of the regression by plotting your predicted values vs target values on a scatter plot, and drawing a line y=x. If all predictions were perfect, the predicted values would lie on the line.

In [ ]:

plt.ylim(20, 350)
plt.xlim(20, 350)
plt.scatter(pred,Yval,color='black')
x = np.linspace(20,350,100)
y=x
plt.plot(x, y,color='blue')

# Extra question: Polynomial regression
The term 'linear' in linear regression refers only to the coefficients $\theta$. We can in fact compute polynomial terms in the data and perform linear regression over this extended dataset to get a better fit to the data.

To compute polynomial terms in the data automatically, you can use the class `sklearn.preprocessing.PolynomialFeatures`. To find out how to use it, look at the guidance (you can type `help(PolynomialFeatures)` once you have imported it).

The following small dataset (in the cell below) gives a relationship between temperature and yield for an experiment. Use cross-validation to select the degree of the polynomial that best fits this data.

Plot the mean squared error against degree on the training set and on the validation set. Which degree of polynomial best fits this data?

In [ ]:
X = np.array([50,50,50,70,70,70,80,80,80,90,90,90,100,100,100]).reshape(-1, 1)
y = np.array([3.3,2.8,2.9,2.3,2.6,2.1,2.5,2.9,2.4,3,3.1,2.8,3.3,3.5,3]).reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
Xtr, Xtest, Ytr, Ytest = train_test_split(X, y, test_size=0.2, random_state=0)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtr, Ytr, test_size=0.2, random_state=0)
mse_tr = []
mse_val = []
max_deg = 10
for i in range(max_deg):
    poly=PolynomialFeatures(degree=i+1)
    Xtrain_new = poly.fit_transform(Xtrain)
    Xval_new = poly.fit_transform(Xval)
    regr.fit(Xtrain_new, Ytrain)
    pred_tr = regr.predict(Xtrain_new)
    pred_v = regr.predict(Xval_new)
    mse_tr.append(mean_squared_error(pred_tr, Ytrain))
    mse_val.append(mean_squared_error(pred_v, Yval))


In [ ]:
plt.plot(range(1, max_deg+1), mse_tr, label='Training')
plt.plot(range(1, max_deg+1), mse_val, label='Validation')
plt.legend()
plt.xlabel('Degree')
plt.ylabel('MSE')

In [ ]:
Xtest_new = poly.fit_transform(Xtest)
pred_test = regr.predict(Xtest_new)
mean_squared_error(pred_test, Ytest)